In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_df = "../output_data/city_data.csv"
weather_df = pd.read_csv(weather_df)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,date
0,Minbu,20.1833,94.8833,76.91,31,0,3.51,MM,1615661466
1,Tuktoyaktuk,69.4541,-133.0374,-13.00,77,90,5.75,CA,1615661467
2,Cape Town,-33.9258,18.4232,72.00,78,75,9.22,ZA,1615661171
3,Bluff,-46.6000,168.3333,46.99,88,4,1.01,NZ,1615661469
4,Kochubey,44.3861,46.5825,28.76,88,17,13.27,RU,1615661471
...,...,...,...,...,...,...,...,...,...
576,Hervey Bay,-25.2986,152.8535,70.00,92,49,7.00,AU,1615661963
577,Forsytheganj,52.6333,29.7333,36.07,95,100,12.95,BY,1615662113
578,Benghazi,32.1167,20.0667,60.17,60,100,15.28,LY,1615662188
579,Tinyahuarco,-10.7500,-76.3000,49.82,71,100,5.82,PE,1615662189


#### Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[['Lat', 'Lng']]
humidity = weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2)

# Add Heatmap layer to map.
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# Create new DataFrame fitting weather criteria
ideal_cond = weather_df.loc[(weather_df['Max Temp']>=70) & 
                            (weather_df['Max Temp']<80) & 
                            (weather_df['Wind Speed']<10) & 
                           (weather_df['Cloudiness'] == 0)]
# Reset Index
ideal_cond = ideal_cond.reset_index()
del ideal_cond['index']
ideal_cond

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,date
0,Port Elizabeth,-33.9180,25.5701,71.60,88,0,4.61,ZA,1615613873
1,Busselton,-33.6500,115.3333,71.01,50,0,4.00,AU,1615614123
2,East London,-33.0153,27.9116,78.80,65,0,8.05,ZA,1615613891
3,Caravelas,-17.7125,-39.2481,71.17,85,0,4.38,BR,1615613913
4,Acapulco de Juárez,16.8634,-99.8901,75.06,71,0,3.15,MX,1615614221
5,Les Cayes,18.2000,-73.7500,72.34,84,0,7.25,HT,1615614384
6,Dali,25.7000,100.1833,72.68,28,0,4.03,CN,1615614410
7,Pochutla,15.7432,-96.4661,75.99,71,0,1.99,MX,1615614572
8,Lázaro Cárdenas,17.9583,-102.2000,71.33,78,0,3.58,MX,1615614637
9,Coahuayana Viejo,18.7333,-103.6833,70.95,53,0,3.27,MX,1615614707


In [16]:
hotel_df = ideal_cond.dropna(how='any')
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,date
0,Port Elizabeth,-33.9180,25.5701,71.60,88,0,4.61,ZA,1615613873
1,Busselton,-33.6500,115.3333,71.01,50,0,4.00,AU,1615614123
2,East London,-33.0153,27.9116,78.80,65,0,8.05,ZA,1615613891
3,Caravelas,-17.7125,-39.2481,71.17,85,0,4.38,BR,1615613913
4,Acapulco de Juárez,16.8634,-99.8901,75.06,71,0,3.15,MX,1615614221
5,Les Cayes,18.2000,-73.7500,72.34,84,0,7.25,HT,1615614384
6,Dali,25.7000,100.1833,72.68,28,0,4.03,CN,1615614410
7,Pochutla,15.7432,-96.4661,75.99,71,0,1.99,MX,1615614572
8,Lázaro Cárdenas,17.9583,-102.2000,71.33,78,0,3.58,MX,1615614637
9,Coahuayana Viejo,18.7333,-103.6833,70.95,53,0,3.27,MX,1615614707


### Hotel Map

In [39]:
# Add a "Hotel Name" column to the DataFrame.
hotels = []

# Loop through cities to find the nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]
    
    city_coord = f'{lat},{lng}'
    
    params = {
        "location": city_coord, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_req = requests.get(base_url, params=params)
    hotel_response = hotel_req.json()
    
    try:
        hotels.append(hotel_response['results'][1]['name'])
    except:
        hotels.append('Not found!')
        
# create dataframe
hotel_df["Hotel Name"] = ""

In [41]:
#print(hotels)
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,date,Hotel Name
0,Port Elizabeth,-33.9180,25.5701,71.60,88,0,4.61,ZA,1615613873,Newtondale Self Catering
1,Busselton,-33.6500,115.3333,71.01,50,0,4.00,AU,1615614123,Restawile Motel
2,East London,-33.0153,27.9116,78.80,65,0,8.05,ZA,1615613891,Woodleigh Guest Lodge
3,Caravelas,-17.7125,-39.2481,71.17,85,0,4.38,BR,1615613913,Pousada Tropical
4,Acapulco de Juárez,16.8634,-99.8901,75.06,71,0,3.15,MX,1615614221,Hotel Bali-Hai
5,Les Cayes,18.2000,-73.7500,72.34,84,0,7.25,HT,1615614384,Hôtel de la Découverte
6,Dali,25.7000,100.1833,72.68,28,0,4.03,CN,1615614410,Hotel Dalifornia
7,Pochutla,15.7432,-96.4661,75.99,71,0,1.99,MX,1615614572,Posada San Jose
8,Lázaro Cárdenas,17.9583,-102.2000,71.33,78,0,3.58,MX,1615614637,Baymont by Wyndham Lazaro Cardenas
9,Coahuayana Viejo,18.7333,-103.6833,70.95,53,0,3.27,MX,1615614707,Hotel Los Arcos


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [44]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))